# Biblioteca QuantLib - Colab em revisão

Veja: https://www.quantlib.org/docs.shtml



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as py
!pip install QuantLib
import QuantLib as ql
import warnings
warnings.filterwarnings('ignore')

  Obtaining dependency information for QuantLib from https://files.pythonhosted.org/packages/04/0c/7b5d362cb326cf5590402c50be1d4e8dbda6fb713050f1b29e2aa03af475/QuantLib-1.32-cp310-cp310-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 5.5 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


## Exemplo de Opções Europeias

In [ ]:
hoje = ql.Date(1, 3, 2023)
vencimento = ql.Date(31, 5, 2023)
#ql.SimpleDayCounter().yearFraction(hoje, vencimento)
ql.Settings.instance().evaluationDate =  hoje
opt_tipo = ql.Option.Call

dc = ql.SimpleDayCounter()  #distâncias em meses completos são retornadas como uma fração simples,
                            #ou seja, 1 ano = 1,0; 6 meses = 0,5; 3 meses = 0,25 e assim por diante.
calendar = ql.NullCalendar() #Ignora feriados - utilizado para reproduzir resultados teóricos

# Veja também, por exemplo:
#calendar = ql.Brazil(ql.Brazil.Settlement) #Settlement: calendário genérico de liquidação; Exchange: calendário BOVESPA
#dc = ql.Business252(calendar)

S0 = 50 #preço atual do ativo objeto
K = 49 #Strike
r = 0.06 #Taxa de juros livre de risco
sigma = 0.3 #volatilidade
d =  0.0 #rendimento de dividendos


In [ ]:
european_exercise = ql.EuropeanExercise(vencimento) #Nas européias, o exercício é no vencimento
payoff = ql.PlainVanillaPayoff(opt_tipo, K) #Payoff
european_option = ql.VanillaOption(payoff, european_exercise) #Opção Vanilla

In [ ]:
r_ts = ql.YieldTermStructureHandle(ql.FlatForward(hoje, r, dc)) #Define a entrada de tx de juros
sigma_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(hoje, calendar, sigma, dc)) #Define a entrada de volatilidade
d_ts = ql.YieldTermStructureHandle(ql.FlatForward(hoje, d, dc)) #Define a entrada de rendimento de dividendos
bsm_process = ql.BlackScholesMertonProcess(ql.QuoteHandle(ql.SimpleQuote(S0)), d_ts, r_ts, sigma_ts) #Cria processo BSM com os parâmetros de entrada

In [ ]:
BS_greek_dict = {} #Cria um dicionário para apresentar as gregas do modelo BSM
Eur_pricing_dict = {} #Cria um dicionário para apresentar as respostas de precificação

bsm = ql.AnalyticEuropeanEngine(bsm_process) #Define o método analalítico
european_option.setPricingEngine(bsm) #Aplica o método analítico para resolver
Eur_pricing_dict['BlackScholesEuropean'] = european_option.NPV() #net present value

BS_greek_dict['Delta'] = european_option.delta()
BS_greek_dict['Gamma'] = european_option.gamma()
BS_greek_dict['Vega'] = european_option.vega()
BS_greek_dict['Theta'] = european_option.theta()
BS_greek_dict['Rho'] = european_option.rho()
print(BS_greek_dict)

# Compara com árvore CRR
crr_engine = ql.BinomialVanillaEngine(bsm_process, "crr", 100) #Define o método CCR com 100 dicretizações
european_option.setPricingEngine(crr_engine) #Aplica o método CRR para resolver
Eur_pricing_dict['CRR'] = european_option.NPV()
print(Eur_pricing_dict)


### Exemplo utilizando o modelo de Heston

O modelo de Heston é uma forma alternativa para avaliar opções. Sua principal característica é a incorporação de um termo de volatilidade estocástica. Ele consiste em dois movimentos Brownianos correlacionados:

$$dS_t = \mu S_t dt + \sqrt{v_t} S_t dW_{1,t}$$

$$dv_t = \kappa(\theta - v_t) dt + \xi \sqrt{v_t} dW_{2,t}$$

A primeira equação representa a evolução do preço do ativo objeto. Já a segunda, representa a evolução da variância instantânea, sendo:


*   $\theta$: termo médio de $v_t$;
*   $\kappa$: velocidade de reversão;
*   $\mu$: taxa de retorno de longo prazo;
*   $\xi$: "volatilidade da volatilidade";
*   $W_{1,t}$: processos de Wiener que governa $S_t$;
*   $W_{2,t}$: processos de Wiener que governa $v_t$.

A correlação entre $W_{1,t}$ e $W_{2,t}$ é dada por $\rho$.

Link do artigo: https://academic.oup.com/rfs/article-abstract/6/2/327/1574747?redirectedFrom=fulltext





In [ ]:
v0 = sigma**2 # variância inicial
kappa = 0.1
theta = v0
xi = 0.1
rho = -0.75
# OBS: tais parâmetros precisam ser calibrados

hest_process = ql.HestonProcess(r_ts, d_ts, ql.QuoteHandle(ql.SimpleQuote(S0)), v0, kappa, theta, xi, rho)
hest_engine = ql.AnalyticHestonEngine(ql.HestonModel(hest_process))
european_option.setPricingEngine(hest_engine)
hest_price = european_option.NPV()
Eur_pricing_dict['Heston'] = european_option.NPV()
print(Eur_pricing_dict)

## Exemplo de Opções Americanas

In [ ]:
hoje = ql.Date(1, 1, 2022)
vencimento = ql.Date(31, 12, 2023)
#ql.SimpleDayCounter().yearFraction(hoje, vencimento)
ql.Settings.instance().evaluationDate =  hoje
opt_tipo = ql.Option.Put

dc = ql.SimpleDayCounter()  #distâncias em meses completos são retornadas como uma fração simples,
                            #ou seja, 1 ano = 1,0; 6 meses = 0,5; 3 meses = 0,25 e assim por diante.
calendar = ql.NullCalendar() #Ignora feriados - utilizado para reproduzir resultados teóricos

S0 = 50 #preço atual do ativo objeto
K = 52 #Strike
r = 0.05 #Taxa de juros livre de risco
sigma = 0.3 #volatilidade
d =  0.0 #rendimento de dividendos

In [ ]:
american_exercise = ql.AmericanExercise(hoje, vencimento) #O exercício pode occorer entre hoje e o vencimento
payoff = ql.PlainVanillaPayoff(opt_tipo, K) #Payoff
american_option = ql.VanillaOption(payoff, american_exercise) #Payoff

In [ ]:
r_ts = ql.YieldTermStructureHandle(ql.FlatForward(hoje, r, dc)) #Define a entrada de tx de juros
sigma_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(hoje, calendar, sigma, dc)) #Define a entrada de volatilidade
d_ts = ql.YieldTermStructureHandle(ql.FlatForward(hoje, d, dc)) #Define a entrada de rendimento de dividendos
bsm_process = ql.BlackScholesMertonProcess(ql.QuoteHandle(ql.SimpleQuote(S0)), d_ts, r_ts, sigma_ts) #Cria processo BSM com os parâmetros de entrada

In [ ]:
Ame_pricing_dict = {} #Cria um dicionário para apresentar as respostas de precificação

#Método de Monte Carlo
N = 200
rng = "pseudorandom"
M = 100000
MC_engine = ql.MCAmericanEngine(bsm_process, rng, N, requiredSamples=M)
american_option.setPricingEngine(MC_engine)
Ame_pricing_dict['MCApproximation'] = american_option.NPV()

#Método das Diferenças Finitas
fd_engine = ql.FdBlackScholesVanillaEngine(bsm_process)
american_option.setPricingEngine(fd_engine)
Ame_pricing_dict['FiniteDifference'] = american_option.NPV()

#Método CRR
crr_engine = ql.BinomialVanillaEngine(bsm_process, "crr", 100) #Define o método CCR com 100 dicretizações
american_option.setPricingEngine(crr_engine) #Aplica o método CRR para resolver
Ame_pricing_dict['CRR'] = american_option.NPV()
print(Ame_pricing_dict)

# Para outros engines, veja: https://quantlib-python-docs.readthedocs.io/en/latest/pricing_engines/options.html